In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, IntSlider, VBox, HBox, interactive_output
from IPython.display import display

# Integer-Constrained Allocation via Parametric Optimization

This notebook implements a **general integer-constrained optimization framework**
for allocating a fixed number of indivisible units across groups of unequal size.

The motivating and only example used here is **representative apportionment**
across a fixed set of groups with known populations.

Although the framework is general, we deliberately restrict attention
to a single concrete case in order to make the modeling assumptions,
objective structure, and tradeoffs fully explicit and auditable.


## Optimization Problem

Given:

- n groups indexed by i = 1,…,n
- population vector P = (P₁,…,Pₙ), with Pᵢ > 0
- a fixed total number of indivisible units k

Choose integer allocations aᵢ such that:

- aᵢ ∈ ℤ
- aᵢ ≥ 1
- ∑ᵢ aᵢ = k

so as to minimize a parametric fairness error E(a).

Optionally, when comparing different values of k,
we introduce a complexity penalty to trade off fairness
against institutional size.


## Modeling Decomposition

The framework separates four concepts that are often conflated:

1. **Target formation**  
   What allocation would be ideal in the continuous limit?

2. **Error measurement**  
   How is deviation from that ideal quantified?

3. **Error aggregation**  
   How much does each group contribute to total unfairness?

4. **Model complexity**  
   How costly is increasing the total number of units?

Each concept is controlled by an independent parameter.


## Parameters

κ (kappa) — Target scaling  
Controls how ideal allocation scales with population.

- κ = 1.0 → proportional targets
- κ = 0.0 → equal targets
- 0 < κ < 1 → diminishing returns to scale

γ (gamma) — Error weighting  
Controls how much each group contributes to total error.

- γ = 1.0 → population-weighted fairness
- γ = 0.0 → equal group weighting

β (beta) — Curvature  
Controls how sharply deviations from ideal are penalized.

- β = 1 → linear penalty
- β > 1 → convex penalty (large deviations dominate)

σ (sigma) — Directional asymmetry  
Controls relative penalty of over- vs under-allocation.

- σ = 0.5 → symmetric
- σ > 0.5 → over-allocation penalized more
- σ < 0.5 → under-allocation penalized more

α (alpha) — Complexity penalty  
Controls the cost of increasing the total number of units k
when comparing different allocations across k.

α does **not** affect within-k optimization.


In [ ]:
def targets_from_kappa(populations, kappa):
    pops = np.asarray(populations, dtype=float)
    t = pops ** kappa
    return t / t.sum()

In [ ]:
def weights_from_gamma(populations, gamma):
    pops = np.asarray(populations, dtype=float)
    w = pops ** gamma
    return w / w.sum()

## Representation Ratios

Let:

- aᵢ = allocated integer units
- aᵢ* = k · Tᵢ = ideal (continuous) allocation
- ϕᵢ = aᵢ / aᵢ*

The objective penalizes deviations from ϕᵢ = 1.

Over- and under-representation are treated separately,
with curvature controlled by β and directional bias by σ.


In [ ]:
def group_error_terms(a, a_star, beta, sigma):
    a = np.asarray(a, dtype=float)
    a_star = np.asarray(a_star, dtype=float)

    phi = a / np.maximum(a_star, 1e-12)
    over = phi >= 1.0

    delta = np.zeros_like(phi)
    delta[over] = (phi[over] - 1.0) ** beta
    delta[~over] = ((1.0 / np.maximum(phi[~over], 1e-12)) - 1.0) ** beta

    multiplier = np.where(over, 2.0 * sigma, 2.0 * (1.0 - sigma))
    return delta, multiplier, phi


def total_error(a, a_star, weights, beta, sigma):
    delta, m, _ = group_error_terms(a, a_star, beta, sigma)
    return np.sum(weights * m * delta)

## Integer Allocation Algorithm

For a fixed k, optimization proceeds as follows:

1. Compute ideal continuous targets aᵢ*
2. Initialize aᵢ = max(1, ⌊aᵢ*⌋)
3. Adjust allocations one unit at a time
4. Each adjustment is chosen to minimize total error E(a)

This is a discrete greedy descent on a smooth,
separable objective under linear constraints.


In [ ]:
def allocate_integer_units(
    populations,
    k,
    kappa=1.0,
    gamma=1.0,
    beta=2.0,
    sigma=0.5
):
    pops = np.asarray(populations, dtype=float)
    n = len(pops)

    if k < n:
        raise ValueError("k must be ≥ number of groups.")

    T = targets_from_kappa(pops, kappa)
    W = weights_from_gamma(pops, gamma)
    a_star = k * T

    a = np.maximum(1, np.floor(a_star)).astype(int)
    diff = int(k - a.sum())

    def E(x):
        return total_error(x, a_star, W, beta, sigma)

    while diff > 0:
        base = E(a)
        best_i, best_gain = None, -np.inf
        for i in range(n):
            cand = a.copy()
            cand[i] += 1
            gain = base - E(cand)
            if gain > best_gain:
                best_gain = gain
                best_i = i
        a[best_i] += 1
        diff -= 1

    while diff < 0:
        base = E(a)
        best_i, best_cost = None, np.inf
        for i in range(n):
            if a[i] <= 1:
                continue
            cand = a.copy()
            cand[i] -= 1
            cost = E(cand) - base
            if cost < best_cost:
                best_cost = cost
                best_i = i
        a[best_i] -= 1
        diff += 1

    return a, E(a), a_star

## Comparing Different Values of k

When k itself is a decision variable, we introduce
a complexity penalty to trade off fairness against size.

Define the score:

S(k) = −log( k^α · E(k) )

- E(k) measures representational error
- k^α penalizes institutional complexity
- α controls how aggressively size is discouraged

The preferred k maximizes S(k).


In [ ]:
def iterate_over_k_parametric(
    population_df,
    k_min,
    k_max,
    alpha=2.0,
    kappa=1.0,
    gamma=1.0,
    beta=2.0,
    sigma=0.5,
    pop_col="Population"
):
    pops = population_df[pop_col].to_numpy(dtype=float)

    ks = []
    allocations = []
    errors = []
    scores = []

    for k in range(int(k_min), int(k_max) + 1):
        a, E, a_star = allocate_integer_units(
            pops, k, kappa=kappa, gamma=gamma, beta=beta, sigma=sigma
        )
        ks.append(k)
        allocations.append(a)
        errors.append(E)
        scores.append(-np.log((k ** alpha) * max(E, 1e-18)))

    return ks, allocations, errors, scores


In [ ]:
##2024 US States Populations
# target_populations_df = pd.DataFrame({
#     "State": [
#         "California", "Texas", "Florida", "New York", "Pennsylvania", "Illinois", "Ohio",
#         "Georgia", "North Carolina", "Michigan", "New Jersey", "Virginia", "Washington",
#         "Arizona", "Tennessee", "Massachusetts", "Indiana", "Maryland", "Missouri",
#         "Wisconsin", "Colorado", "Minnesota", "South Carolina", "Alabama", "Louisiana",
#         "Kentucky", "Oregon", "Oklahoma", "Connecticut", "Utah", "Nevada", "Iowa",
#         "Arkansas", "Kansas", "Mississippi", "New Mexico", "Nebraska",
#         "Idaho", "West Virginia", "Hawaii", "New Hampshire", "Maine", "Montana",
#         "Rhode Island", "Delaware", "South Dakota", "North Dakota", "Alaska",
#         "Vermont", "Wyoming"
#     ],
#     "Population": [
#         39431263, 31290831, 23372215, 19867248, 13078751, 12710158, 11883304, 11180878,
#         11046024, 10140459, 9500851, 8811195, 7958180, 7582384, 7227750, 7136171, 6924275,
#         6263220, 6245466, 5960975, 5957493, 5793151, 5478831, 5157699, 4597740, 4588372,
#         4272371, 4095393, 3675069, 3503613, 3267467, 3241488, 3088354, 2970606,
#         2943045, 2130256, 2005465, 2001619, 1769979, 1446146, 1409032, 1405012, 1137233,
#         1112308, 1051917, 924669, 796568, 740133, 648493, 587618
#     ]
# })



###2020 US States Populations
# target_populations_df = pd.DataFrame({
#     "State": [
#         "California", "Texas", "Florida", "New York", "Pennsylvania", "Illinois", "Ohio",
#         "Georgia", "North Carolina", "Michigan", "New Jersey", "Virginia", "Washington",
#         "Arizona", "Tennessee", "Massachusetts", "Indiana", "Maryland", "Missouri",
#         "Wisconsin", "Colorado", "Minnesota", "South Carolina", "Alabama", "Louisiana",
#         "Kentucky", "Oregon", "Oklahoma", "Connecticut", "Utah", "Nevada", "Iowa",
#         "Arkansas", "Kansas", "Mississippi", "New Mexico", "Nebraska",
#         "Idaho", "West Virginia", "Hawaii", "New Hampshire", "Maine", "Montana",
#         "Rhode Island", "Delaware", "South Dakota", "North Dakota", "Alaska",
#         "Vermont", "Wyoming"
#     ],
#     "Population": [
#         39538223, 29145505, 21538187, 20201249, 13002700, 12812508, 11799448,
#         10711908, 10439388, 10077331, 9288994, 8631393, 7705281,
#         7151502, 6910840, 7029917, 6785528, 6177224, 6154913,
#         5893718, 5773714, 5706494, 5118425, 5024279, 4657757,
#         4505836, 4237256, 3959353, 3605944, 3271616, 3104614, 3190369,
#         3011524, 2937880, 2961279, 2117522, 1961504,
#         1839106, 1793716, 1455271, 1377529, 1362359, 1084225,
#         1097379, 989948, 886667, 779094, 733391,
#         643077, 576851
#     ]
# })

# ###NEW ENGLAND
target_populations_df = pd.DataFrame({"State":[ "Massachusetts","Connecticut","New Hampshire", "Maine","Rhode Island","Vermont"],"Population":[7136171,3675069,1409032,1405012,1112308,648493]})


In [ ]:
target_populations_df

,State,Population
0,Massachusetts,7136171
1,Connecticut,3675069
2,New Hampshire,1409032
3,Maine,1405012
4,Rhode Island,1112308
5,Vermont,648493


In [ ]:
def plot_vs_k(x, y, title, xlabel, ylabel):
    """
    Simple, reliable plotting helper for Colab.

    Parameters
    ----------
    x : list or array
        k values
    y : list or array
        metric values (E(k) or S(k))
    title : str
    xlabel : str
    ylabel : str
    """
    plt.figure(figsize=(8, 5))
    plt.plot(x, y, marker="o")
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def allocation_audit_df(df, a, k, kappa, gamma, beta, sigma,
                        name_col=None, pop_col="Population"):
    df = df.copy()

    if name_col is None:
        # pick a reasonable name column
        for c in ["State", "Group", "Name"]:
            if c in df.columns:
                name_col = c
                break
        if name_col is None:
            name_col = df.columns[0]

    pops = df[pop_col].to_numpy(dtype=float)
    a = np.asarray(a, dtype=int)

    T = targets_from_kappa(pops, kappa)
    W = weights_from_gamma(pops, gamma)
    a_star = k * T

    delta, m, _ = group_error_terms(a, a_star, beta, sigma)
    contrib = W * m * delta

    out = pd.DataFrame({
        name_col: df[name_col].values,
        pop_col: pops,
        "Ideal a*": np.round(a_star, 3),
        "Allocated a": a,
        "Ratio φ": np.round(a / np.maximum(a_star, 1e-18), 4),
        "Over/Under": np.where(a >= a_star, "over", "under"),
        "Weighted contrib": np.round(contrib, 10),
    })

    # clean, useful default ordering: biggest drivers of error first
    return out.sort_values("Weighted contrib", ascending=False).reset_index(drop=True)


def update_and_plot(alpha, beta, sigma, kappa, gamma, k_min, k_max):
    ks, allocations, errors, scores = iterate_over_k_parametric(
        target_populations_df,
        k_min=int(k_min),
        k_max=int(k_max),
        alpha=float(alpha),
        kappa=float(kappa),
        gamma=float(gamma),
        beta=float(beta),
        sigma=float(sigma),
        pop_col="Population"
    )

    plot_vs_k(ks, errors, "Error E(k)", "k", "E(k)")
    plot_vs_k(ks, scores, "Score S(k)", "k", "S(k)")

    best_idx = int(np.argmax(scores))
    best_k = ks[best_idx]
    best_a = allocations[best_idx]

    print("Best k:", best_k)
    print("E(k):", errors[best_idx])
    print("S(k):", scores[best_idx])

    display(allocation_audit_df(
        target_populations_df, best_a, best_k,
        kappa=float(kappa), gamma=float(gamma),
        beta=float(beta), sigma=float(sigma),
        pop_col="Population"
    ))

In [ ]:
# Nice compact label alignment (optional but recommended)
STYLE = {'description_width': '32px'}

α = FloatSlider(value=2.0, min=0.0, max=6.0, step=0.1, description='α', style=STYLE)
β = FloatSlider(value=2.0, min=1.0, max=6.0, step=0.1, description='β', style=STYLE)
σ = FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description='σ', style=STYLE)
κ = FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description='κ', style=STYLE)
γ = FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description='γ', style=STYLE)

kₘᵢₙ = IntSlider(value=6,  min=len(target_populations_df), max=500,  step=1, description='kₘᵢₙ', style={'description_width': '48px'})
kₘₐₓ = IntSlider(value=60, min=len(target_populations_df), max=1000, step=1, description='kₘₐₓ', style={'description_width': '48px'})

ui = VBox([
    HBox([α, β, σ,κ, γ]),
    HBox([kₘᵢₙ, kₘₐₓ]),
])

out = interactive_output(update_and_plot, {
    'alpha': α,
    'beta':  β,
    'sigma': σ,
    'kappa': κ,
    'gamma': γ,
    'k_min': kₘᵢₙ,
    'k_max': kₘₐₓ
})

display(ui, out)